# Firestore PoC

Demonstrates python functionality for taking data from races and aggregating it in the appropriate collections in Firestore. Can also be used to add sample data to our database.

## Initialize Cloud Firestore SDK on Google Cloud

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

env_path = Path('..')/'.env'
load_dotenv(dotenv_path=env_path)

if not firebase_admin._apps:
    cred = credentials.Certificate(os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

## Utility Functions

In [2]:
import re

def getDocId(name: str):
    cleaned = re.sub(r"[/?*=#]", "", name)
    return cleaned.replace(" ", "_").lower()

## Create Sample Race Data

In [3]:
track = "Baby Park"
eventName = "Trumbull Circuit #1"

print(getDocId(eventName))

trumbull_circuit_1
